<a href="https://colab.research.google.com/github/PrakharDoneria/Grey-Hat-LLM/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# ⚙️ Install necessary libraries
!pip install -q transformers accelerate bitsandbytes

import torch
if torch.cuda.is_available():
    print("✅ CUDA is available. Using GPU.")
else:
    print("⚠️ CUDA not available. Using CPU (will be slow).")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.6 MB/s eta 0:00:00
✅ CUDA is available. Using GPU.


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "WhiteRabbitNeo/WhiteRabbitNeo-V3-7B"

# 🧠 Load tokenizer and model (full precision, no quantization)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",         # Uses FP16 on supported GPUs
    device_map="auto"           # Automatically selects GPU if available
)

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [10]:
from transformers import pipeline

# 🧠 Create text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [13]:
from transformers import TextIteratorStreamer
import threading

# 📝 Real-time streaming response function
def generate_response_streaming(prompt, max_new_tokens=256, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 📺 Setup the streaming output
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # 🔄 Run model.generate() in a thread so it doesn't block streaming
    generation_kwargs = dict(
        inputs=inputs.input_ids,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        streamer=streamer
    )
    thread = threading.Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # 🧠 Print tokens as they arrive
    print("Response:\n", end="")
    for new_text in streamer:
        print(new_text, end="", flush=True)

# 🧪 Try a test prompt
test_prompt = "How can a penetration tester find open ports on a target system?"
print("Prompt:", test_prompt)
generate_response_streaming(test_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Prompt: How can a penetration tester find open ports on a target system?
Response:
 To find open ports on a target system using a penetration testing approach, you can utilize various tools and techniques. Here are some effective methods:

### Nmap
Nmap is one of the most popular and powerful port scanning tools available. It performs TCP connect scans by default, which attempt to establish a full TCP connection to each port.

To use Nmap for port scanning:
1. Install Nmap if not already installed
2. Run the basic command to scan all ports:
   ```bash
   nmap -p- <target_ip>
   ```
   The `-p-` flag tells Nmap to scan all ports (1 to 65535).

3. To scan specific ports only:
   ```bash
   nmap -p 80,443 <target_ip>
   ```

4. For faster scanning:
   ```bash
   nmap -sS -p 1-1000 <target_ip>
   ```
   The `-sS` flag performs a SYN scan which is faster than connect scans.

5. For verbose output:
   ```bash
   nmap -v -p 80,443 <target_ip>
   ```
   The `-v` flag increases verbosity.

###